# Database Test Notebook

In [1]:
%pip install -q mysql-connector-python psycopg2-binary graphviz ibis-framework

Note: you may need to restart the kernel to use updated packages.


### References

- https://www.geeksforgeeks.org/how-to-connect-python-with-sql-database/#
- https://www.datacamp.com/tutorial/tutorial-postgresql-python
- https://stackoverflow.com/questions/12047193/how-to-convert-sql-query-result-to-pandas-data-structure
- https://stackoverflow.com/questions/43633891/validating-a-data-type-in-python

In [1]:
import os
import mysql.connector
import psycopg2
import pandas as pd
import graphviz


## Connect to the Database

Follows the Python Database API Specification v2.0

Source: https://peps.python.org/pep-0249/

### MySQL

In [ ]:
# Creating MySQL connection object
mydb = mysql.connector.connect(
    database = os.environ['DB_NAME'],
    host = os.environ['DB_HOST'],
    user = os.environ['DB_USER'],
    password = os.environ['DB_PASSWORD']
)

### Postgres

In [2]:
mydb = psycopg2.connect(
    database = os.environ['POSTGRES_DB_NAME'],
    host = os.environ['POSTGRES_DB_HOST'],
    user = os.environ['POSTGRES_DB_USER'],
    password = os.environ['POSTGRES_DB_PASSWORD']
)

#### Create Cursor

In [5]:
# Creating an instance of 'cursor' class which is used to execute the 'SQL' statements in 'Python'
cursor = mydb.cursor()

## Queries

In [ ]:
query = [
    "SELECT distinct(meta_key) FROM vce_users_meta",
    "SELECT * FROM vce_users LIMIT 20",
    "SELECT * FROM vce_users_meta LIMIT 20",
    "SELECT distinct(role_id) FROM vce_users",
    "SELECT meta_value FROM vce_site_meta WHERE meta_key='roles'",
    "SELECT distinct(meta_key) FROM vce_users_meta",
    "SELECT * FROM monolithic.vce_components",
]

In [3]:
query = [
    """select distinct(b.meta_value) from public.vce_components_meta a join public.vce_components_meta b on a.component_id = b.component_id and b.meta_key='type' and a.meta_key not like 'lms_assignment_id%';"""
]

### MySQL/Psycopg2 Connector Method

In [6]:
cursor.execute(query[0])# Print contents of the cursor
for x in cursor:
  print(x)

('Access',)
('AccreditationCertificate',)
('AccreditationSubmission',)
('ActionPlans',)
('AddPlayByPlay',)
('Alias',)
('Annotations',)
('AssessmentsClassroom',)
('AssessmentsDashbaord',)
('AssessmentsEvaluation',)
('AssessmentsLister',)
('AssessmentsRating',)
('AssessmentsReport',)
('AssessmentsReportClassroom',)
('AssessmentsSite',)
('Assets',)
('Assignments',)
('AWSDashboard',)
('CoachingPartnership',)
('CollaboraEndpoint',)
('Comments',)
('ConnectToMerit',)
('ContactForm',)
('CopAssets',)
('CopGroups',)
('CopLocation',)
('CoursesState',)
('CycleDashboard',)
('DiscussionBoards',)
('DiscussionTopics',)
('Documentation',)
('DualLanguageHandbook',)
('DualLanguageSubmission',)
('Ea_eval_survey',)
('Endpoint',)
('FamilySurveyLink',)
('Grade',)
('Inquisitor',)
('InstructorMaterials',)
('Item',)
('Layout',)
('Limit',)
('Links',)
('Lister',)
('Location',)
('Login',)
('Logout',)
('LtiEndpoint',)
('ManageComponents',)
('ManageDatalists',)
('ManageGroups',)
('ManageMenus',)
('ManageRecipes',)
(

#### Pandas Method

In [7]:
df = pd.read_sql(query[0], con = mydb)
# print(df)
# parent_id_list = df['parent_id'].unique().tolist()
# parent_id_list.sort(key=int) # List of unique parent_id values
# print(parent_id_list)

/tmp/ipykernel_5600/3708841152.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query[0], con = mydb)


In [ ]:
%pip install -q graphviz

In [ ]:
w = graphviz.Digraph('wide')

for index, row in df.iterrows():
 w.edge(str(row["component_id"]), str(row["parent_id"]), label='')

w.view()

# w.render('vce_components.gv.pdf', view=True)  # doctest: +SKIP

## Using Ibis

In [2]:
import os
import ibis

In [3]:
# Establish Connection
conn = ibis.postgres.connect(
    host=os.environ['POSTGRES_DB_HOST'],
    user=os.environ['POSTGRES_DB_USER'],
    password=os.environ['POSTGRES_DB_PASSWORD'],
    database=os.environ['POSTGRES_DB_NAME']
)

In [4]:
# List all tables in the database
conn.list_tables()

['auth_group',
 'auth_group_permissions',
 'auth_permission',
 'auth_user',
 'auth_user_groups',
 'auth_user_user_permissions',
 'django_admin_log',
 'django_content_type',
 'django_migrations',
 'django_session',
 'django_site',
 'dra_auth_systems',
 'dra_crud_permissions',
 'dra_j_cecu_roles_to_org_uids',
 'dra_j_org_subdivisions_to_org_uids',
 'dra_j_org_uids_to_temp_emails',
 'dra_j_org_uids_to_temp_users',
 'dra_j_org_usernames_to_org_uids',
 'dra_j_temp_users_to_temp_subgroups',
 'dra_j_temp_users_to_temp_user_attributes',
 'dra_j_to_temp_content_to_org_uids_to_ceci_roles_to_temp_subgrou',
 'dra_j_uids_to_org_subdivisions_to_users',
 'dra_j_uids_to_orgs_to_users',
 'dra_j_users_to_roles',
 'dra_org_subdivisions',
 'dra_org_usernames',
 'dra_organizations',
 'dra_roles',
 'dra_temp_access_control_rules',
 'dra_temp_content',
 'dra_temp_emails',
 'dra_temp_groups',
 'dra_temp_subgroups',
 'dra_temp_user_attributes',
 'dra_temp_users',
 'dra_uids',
 'ea_access_permissions',
 'lti1p3

In [6]:
conn.list_tables()[0]
len(conn.list_tables())

65

In [6]:
# Load a table
vce_components_meta = conn.table('vce_components_meta')
vce_components_meta.columns

['id', 'component_id', 'meta_key', 'meta_value', 'minutia']

In [7]:
# Assuming vce_components_meta is already defined as conn.table('vce_components_meta')

# Alias for self-join
a = vce_components_meta.alias('a')  # Add alias for table a
b = vce_components_meta.alias('b')  # Add alias for table b

# Perform the join with conditions
# Correcting the filter condition for a.meta_key not like 'lms_assignment_id%'
joined = a.join(b, [a.component_id == b.component_id,
					 b.meta_key == 'type',
					 ~a.meta_key.like('lms_assignment_id%')])

# Select distinct b.meta_value
query = joined[b.meta_value].distinct()

# Execute the query
result = query.execute()

# Print the result
print(result)

                  meta_value
0                     Access
1   AccreditationCertificate
2    AccreditationSubmission
3                ActionPlans
4              AddPlayByPlay
..                       ...
91           UsersLogUploads
92           UserSubmissions
93           VideoConference
94  VideoHighlightSubmission
95           VideoReflection

[96 rows x 1 columns]


In [8]:
# Assuming conn is your Ibis connection to the database
vce_components_meta = conn.table('vce_components_meta')

# Aliases for self-join
a = vce_components_meta.alias('a')
b = vce_components_meta.alias('b')
c = vce_components_meta.alias('c')

# Perform the joins
joined = a.join(b, a.component_id == b.component_id) \
		   .join(c, [a.component_id == c.component_id,
					 c.meta_key == 'type',
					 c.meta_value == 'AWSDashboard',
					 ~a.meta_key.like('lms_assignment_id%')])

# Select distinct b.meta_key
query = joined[b.meta_key].distinct()

# Execute the query
result = query.execute()

print(result)

     meta_key
0  created_at
1  created_by
2       title
3        type


In [9]:
import ibis
import yaml

def extract_classes_and_attributes(conn, table_name):
	vce_components_meta = conn.table(table_name)
	
	# Aliases for self-join
	a = vce_components_meta.alias('a')
	b = vce_components_meta.alias('b')
	c = vce_components_meta.alias('c')
	
	# Query for distinct classes
	class_query = a.join(b, [a.component_id == b.component_id,
							 b.meta_key == 'type',
							 ~a.meta_key.like('lms_assignment_id%')])[b.meta_value].distinct()
	classes = class_query.execute()
	
	# Dictionary to hold class attributes
	class_attributes = {}
	
	for class_name in classes:
		# Query for attributes of the current class
		attribute_query = a.join(b, a.component_id == b.component_id) \
						   .join(c, [a.component_id == c.component_id,
									 c.meta_key == 'type',
									 c.meta_value == class_name,
									 ~a.meta_key.like('lms_assignment_id%')])[b.meta_key].distinct()
		attributes = attribute_query.execute()
		
		# Store attributes for the current class
		class_attributes[class_name] = attributes.tolist()
	
	# Save the results as a YAML file
	with open('class_attributes.yaml', 'w') as file:
		yaml.dump(class_attributes, file, default_flow_style=False)

# Example usage
# Assuming 'conn' is your Ibis connection
# extract_classes_and_attributes(conn, 'vce_components_meta')